In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_staging.stg_systems
USING DELTA
AS
SELECT
  time,
  systems.equipment_id,
  systems.im_id,
  systems.component.columns.column_id
FROM workspace.silver_schema.silver_delta_table;

MERGE INTO workspace.gold_staging.stg_systems AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      systems.equipment_id,
      systems.im_id,
      systems.component.columns.column_id,
      ROW_NUMBER() OVER (
        PARTITION BY time, systems.equipment_id  -- dedup by key columns
        ORDER BY time DESC                       -- keep latest (or earliest) row
      ) AS rn
    FROM workspace.silver_schema.silver_delta_table
  ) WHERE rn = 1
) AS source
ON target.equipment_id = source.equipment_id
AND target.time = source.time
WHEN MATCHED THEN
  UPDATE SET
    target.im_id = source.im_id,
    target.column_id = source.column_id
WHEN NOT MATCHED THEN
  INSERT (time, equipment_id, im_id, column_id)
  VALUES (source.time, source.equipment_id, source.im_id, source.column_id);
